# Task 5 - Tests

In [32]:
from helpers.cellular_automaton import fill_from_scenario_file, CellularAutomaton

## TEST1: RiMEA scenario 1 (straight line, ignore premovement time)

It is to be proven that a person in a 2 m wide and 40 m long corridor with a defined walking speed will cover the distance in the corresponding time period.
If 40 cm (body dimension), 1 second (premovement time) and 5% (walking speed) are set as imprecise values, then the following requirement results with a typical pedestrian speed of 1.33 m/sec: the speed should be set at a value between 4.5 and 5.1 km/h. The travel time should lie in the range of 26 to 34 seconds when 1.33 m/sec is set as the speed.

In [33]:
cellularAutomaton_task5_rimea1 = fill_from_scenario_file('scenario_files/tasks/scenario_task_5_rimea_1.csv')
cellularAutomaton_task5_rimea1.visualize_state_grid()

[P                                                                                                                    T]
[                                                                                                                      ]


In [34]:
cellularAutomaton_task5_rimea1.simulate_until_no_change()
cellularAutomaton_task5_rimea1.visualize_simulation()

Iteration 0:
[P                                                                                                                    T]
[                                                                                                                      ]

Iteration 1:
[   P                                                                                                                 T]
[                                                                                                                      ]

Iteration 2:
[         P                                                                                                           T]
[                                                                                                                      ]

Iteration 3:
[            P                                                                                                        T]
[                                                                                                 

In [35]:
finished_ped = cellularAutomaton_task5_rimea1.finished_pedestrians[0]
finished_ped

{'speed_desired': 1.33,
 'start_pos': (0, 0),
 'curr_pos': (0, 39),
 'travelled': 39.0,
 'skips': 0,
 'finish_iteration': 29}

In [36]:
print(f'Travel time: {finished_ped["finish_iteration"]} s')
print(f'Distance travelled: {finished_ped["travelled"]} m')
print(f'Speed Desired: {finished_ped["speed_desired"]} m/s')
print(f'Speed Achieved: {finished_ped["travelled"] / finished_ped["finish_iteration"]:.2f} m/s')

Travel time: 29 s
Distance travelled: 39.0 m
Speed Desired: 1.33 m/s
Speed Achieved: 1.34 m/s


## TEST2: RiMEA scenario 4 (fundamental diagram, be careful with periodic boundary conditions)

The figure shown below (Figure 4) is to be modelled (corridor 1000 m long, 10 m wide). There are three measuring points (2 x 2 m), the dotted of which is the main measuring point, with the other two grey measuring points functioning as control measuring points.

The corridor is to be filled with different densities of persons with an equal as possible free walking speed (for example 1.2 – 1.4m/s): 0.5 P/m², 1 P/m², 2 P/m², 3 P/m², 4 P/m², 5 P/m² and 6 P/m².

At the measuring points, the average speed of per-sons over a period of 60 seconds is to be deter-mined for the specified density. The first 10 sec-onds can be ignored as a “transient response”. From the results (speed at the specified density), the corresponding fundamental diagrams can be created, with the calculation flow = speed * density taken as a basis for the flow of persons.

In order to ensure that the fundamental diagram is also reproduced by the program in case of a “line movement”, the corridor should be reduced in width to such a degree that the persons can only move one behind the other and overtaking is not possible.

![corridor](img/rimea_4_corridor.png)

In [37]:
# TODO

## TEST3: RiMEA scenario 6 (movement around a corner)
Twenty persons moving towards a corner which turns to the left (Figure 5) will successfully go around it without passing through walls.

![Crossway](img/rimea_6_crossway.png)

Assumptions:
- 1 cell = 0.5 m
- 1 person = 1 cell

In [38]:
cellularAutomaton_task5_rimea6 = fill_from_scenario_file('scenario_files/tasks/scenario_task_5_rimea_6.csv')
cellularAutomaton_task5_rimea6.visualize_state_grid()

[                                                         O  T  T  T  T]
[                                                         O            ]
[                                                         O            ]
[                                                         O            ]
[                                                         O            ]
[                                                         O            ]
[                                                         O            ]
[                                                         O            ]
[                                                         O            ]
[                                                         O            ]
[                                                         O            ]
[                                                         O            ]
[                                                         O            ]
[                                                  

In [39]:
cellularAutomaton_task5_rimea6.simulate_until_no_change()
cellularAutomaton_task5_rimea6.visualize_simulation()

Iteration 0:
[                                                         O  T  T  T  T]
[                                                         O            ]
[                                                         O            ]
[                                                         O            ]
[                                                         O            ]
[                                                         O            ]
[                                                         O            ]
[                                                         O            ]
[                                                         O            ]
[                                                         O            ]
[                                                         O            ]
[                                                         O            ]
[                                                         O            ]
[                                     

## TEST4: RiMEA scenario 7 (demographic parameters, visual comparison of figure and results is sufficient. Simple and correct statistical test gives 5 bonus points)
Select a group consisting of adult persons in accordance with Figure 2 and distribute the walking speeds over a population of 50 persons. Show that the distribution of walking speeds in the simulation is consistent with the distribution in the table.

![Walking speed over age](img/rimea_7_speed_age.png)

In [40]:
# define grid
height, width = 50, 100
cellularAutomaton_task5_rimea7 = CellularAutomaton((height, width))

# place targets at the very end
for i in range(height):
    cellularAutomaton_task5_rimea7.add_target((i, width-1))

# TODO place 50 pedestians with age according to figure on leftmost column.

# visualize
cellularAutomaton_task5_rimea7.visualize_state_grid()


[                                                                                                                                                                                                                                                                                                         T]
[                                                                                                                                                                                                                                                                                                         T]
[                                                                                                                                                                                                                                                                                                         T]
[                                                                                                